In [1]:
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import dados

## Localização de chamados do 1746
#### Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.

1. Quantos chamados foram abertos no dia 01/04/2023?

Resposta: No dia 01/04/2023 foram abertos 1756 chamados.

In [2]:
df_chamado_1746 = dados.chamado_1746()

Downloading: 100%|██████████| 1634336/1634336 [03:23<00:00, 8021.34rows/s]


In [3]:
df_chamado_1o_de_abril = df_chamado_1746[
    (datetime(2023, 4, 1) <= df_chamado_1746["data_inicio"])
    & (df_chamado_1746["data_inicio"] < datetime(2023, 4, 2))
]
df_chamado_1o_de_abril["id_chamado"].count()

1756

2. Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

Resposta: no dia 01/04/2023 o tipo que teve mais chamados abertos foi "Estacionamento irregular".

In [4]:
df_chamado_1o_de_abril[["id_tipo", "tipo"]].value_counts().sort_values(
    ascending=False
).head(1)

id_tipo  tipo                    
782      Estacionamento irregular    366
dtype: int64

3. Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

Resposta: Campo Grande com 113 chamados, Tijuca com 89 e Barra da Tijuca com 59.

In [5]:
df_bairro = dados.bairro()

In [6]:
df_top_reclamacoes = pd.DataFrame(
    df_chamado_1o_de_abril[["id_bairro"]].value_counts(), columns=["count"]
)
pd.merge(
    df_top_reclamacoes,
    df_bairro[["id_bairro", "nome"]].set_index("id_bairro"),
    on=["id_bairro"],
).head(3)

,count,nome
id_bairro,,
144,113,Campo Grande
33,89,Tijuca
128,59,Barra da Tijuca


4. Qual o nome da subprefeitura com mais chamados abertos nesse dia?

Resposta: Zona Norte com 510 chamados.

In [7]:
pd.merge(
    df_top_reclamacoes, df_bairro[["id_bairro", "subprefeitura"]], on=["id_bairro"]
).groupby(["subprefeitura"]).sum().sort_values(by=["count"], ascending=False).head(1)

,count
subprefeitura,
Zona Norte,510


5. Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

Resposta: Existem vários, todos estão associados a transporte/mobilidade ou atendimento por canais eletrônicos.

In [8]:
df_chamado_bairro = pd.merge(
    df_chamado_1o_de_abril[["id_bairro", "tipo", "subtipo"]],
    df_bairro[["id_bairro", "nome", "subprefeitura"]].set_index("id_bairro"),
    on=["id_bairro"],
    how="left",
)
df_chamado_bairro[df_chamado_bairro["nome"].isnull()][
    ["tipo", "subtipo"]
].drop_duplicates()

,tipo,subtipo
68,Ônibus,Fiscalização de irregularidades em linha de ôn...
80,Ônibus,Verificação de ar condicionado inoperante no ô...
210,Estacionamento irregular,Fiscalização de estacionamento irregular de ve...
269,Transporte Especial Complementar - TEC,Fiscalização do transporte complementar
272,Atendimento ao cidadão,Solicitação de correção de falhas e de cadastr...
463,Alvará,Solicitação de orientações sobre o alvará pela...
556,Defesa do Consumidor,Verificação de problemas com produtos ou servi...
868,Atendimento ao cidadão,Solicitação da gravação do atendimento 1746
1027,BRT (corredor expresso de ônibus),Verificação do serviço BRT - Transolímpica


## Chamados do 1746 em grandes eventos
#### Utilize a tabela de Chamados do 1746 e a tabela de Ocupação Hoteleira em Grandes Eventos no Rio para as perguntas de 6-10. Para todas as perguntas considere o subtipo de chamado "Perturbação do sossego".

6. Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

Resposta: 42830 chamados

In [9]:
df_perturbacao = df_chamado_1746[df_chamado_1746["subtipo"] == "Perturbação do sossego"]
df_perturbacao[["id_chamado"]].count()

id_chamado    42830
dtype: int64

7. Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [10]:
df_eventos = dados.eventos()
df_eventos["data_inicial"] = pd.to_datetime(df_eventos["data_inicial"])
df_eventos["data_final"] = pd.to_datetime(df_eventos["data_final"]) + timedelta(days=1)

In [11]:
df_eventos_cross_perturbacao = pd.merge(df_eventos, df_perturbacao, how="cross")
df_eventos_perturbacao = df_eventos_cross_perturbacao.query(
    "data_inicial <= data_inicio < data_final"
)[
    [
        "evento",
        "id_chamado",
        "data_inicial",
        "data_inicio",
        "data_final",
        "id_bairro",
        "id_tipo",
        "tipo",
        "id_subtipo",
        "subtipo",
    ]
]
df_eventos_perturbacao

,evento,id_chamado,data_inicial,data_inicio,data_final,id_bairro,id_tipo,tipo,id_subtipo,subtipo
7452,Rock in Rio,17684216,2022-09-08,2022-09-10 20:50:59,2022-09-12,42,1615,Perturbação do sossego,5071,Perturbação do sossego
7453,Rock in Rio,17684125,2022-09-08,2022-09-10 20:04:05,2022-09-12,43,1615,Perturbação do sossego,5071,Perturbação do sossego
7458,Rock in Rio,17684038,2022-09-08,2022-09-10 18:54:32,2022-09-12,151,1615,Perturbação do sossego,5071,Perturbação do sossego
7462,Rock in Rio,17685088,2022-09-08,2022-09-11 15:09:19,2022-09-12,68,1615,Perturbação do sossego,5071,Perturbação do sossego
7466,Rock in Rio,17684132,2022-09-08,2022-09-10 20:06:37,2022-09-12,50,1615,Perturbação do sossego,5071,Perturbação do sossego
...,...,...,...,...,...,...,...,...,...,...
169749,Carnaval,18328421,2023-02-18,2023-02-18 12:05:37,2023-02-22,64,1615,Perturbação do sossego,5071,Perturbação do sossego
169754,Carnaval,18331560,2023-02-18,2023-02-20 09:09:39,2023-02-22,52,1615,Perturbação do sossego,5071,Perturbação do sossego
169759,Carnaval,18333678,2023-02-18,2023-02-21 11:07:57,2023-02-22,144,1615,Perturbação do sossego,5071,Perturbação do sossego
169761,Carnaval,18331051,2023-02-18,2023-02-19 20:07:53,2023-02-22,14,1615,Perturbação do sossego,5071,Perturbação do sossego


8. Quantos chamados desse subtipo foram abertos em cada evento?

Resposta: 834 no Rock in Rio, 241 no Carnaval, 139 no Reveillon.

In [27]:
df_eventos_perturbacao_count = df_eventos_perturbacao[["evento", "id_chamado"]].drop_duplicates().groupby(
    ["evento"]
).count().sort_values(by=["id_chamado"], ascending=False)
df_eventos_perturbacao_count.head()

,id_chamado
evento,
Rock in Rio,834
Carnaval,241
Reveillon,139


9. Qual evento teve a maior média diária de chamados abertos desse subtipo?

Resposta: Rock in Rio com aproximadamente 119,14 chamados/dia.

In [39]:
df_eventos["duracao"] = (df_eventos["data_final"] - df_eventos["data_inicial"]).apply(
    lambda td: td.days
)
df_eventos_duracao = df_eventos.groupby(["evento"]).sum()[["duracao"]]
df_eventos_resumo = pd.merge(
    df_eventos_duracao,
    df_eventos_perturbacao_count,
    on=["evento"],
    how="inner"
)
df_eventos_resumo["chamados_por_dia"] = df_eventos_resumo["id_chamado"] / df_eventos_resumo["duracao"]
df_eventos_resumo.head()

,duracao,id_chamado,chamados_por_dia
evento,,,
Carnaval,4,241,60.250000
Reveillon,3,139,46.333333
Rock in Rio,7,834,119.142857


10. Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

Resposta: A média no Intervalo de 01/01/2022 a 31/12/2023 (aprox 59,75 chamados/dia) é menor que as de Rock in Rio (aprox 119,14 chamados/dia), do Carnaval (60,25 chamados/dia) e maior que a do Reveillon (aprox 46,33 chamados/dia).

In [60]:
total_chamados = df_perturbacao[["id_chamado"]].count().values[0]
df_eventos_media = df_eventos_resumo[["chamados_por_dia"]]
df_eventos_media.append(
    pd.DataFrame(
        columns=["chamados_por_dia"],
        data=[total_chamados / (date(2023, 12, 31) - date(2022, 1, 1)).days + 1],
        index=["Intervalo de 01/01/2022 a 31/12/2023"],
    )
).sort_values(by=["chamados_por_dia"], ascending=False)
# df_eventos_media

,chamados_por_dia
Rock in Rio,119.142857
Carnaval,60.250000
Intervalo de 01/01/2022 a 31/12/2023,59.751715
Reveillon,46.333333
